In [1]:
import random
import json
import pickle
import numpy as np

In [3]:
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

In [5]:
lemmatizer = WordNetLemmatizer()

In [9]:
import json
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# Load intents with UTF-8 encoding
with open('intents.json', 'r', encoding='utf-8') as file:
    intents = json.load(file)



In [11]:
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [13]:
def clean_sentence(sentence):
    return [lemmatizer.lemmatize(word.lower()) for word in nltk.word_tokenize(sentence)]

In [15]:
def bag_of_words(sentence):
    sentence_words = clean_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [17]:
def predict_intent(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return [{"intent": classes[r[0]], "probability": str(r[1])} for r in results]

In [19]:
def get_response(intent_list):
    tag = intent_list[0]['intent']
    for intent in intents['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])
    return "I didn't understand that. Can you rephrase?"

In [21]:
print("🤖 GradieBears Chatbot is ready! (Type 'quit' to exit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break
    
    ints = predict_intent(user_input)
    res = get_response(ints)
    print("Bot:", res)

🤖 GradieBears Chatbot is ready! (Type 'quit' to exit)


You:  quit
